In [73]:
from glob import glob
import pandas as pd
import numpy as np
import csv
import os
%pylab inline
import matplotlib.pyplot as plt
from subprocess import check_output
from subprocess import Popen, PIPE
import shutil
from nipype.interfaces import fsl
from nipype.interfaces.fsl import RobustFOV, Reorient2Std

Populating the interactive namespace from numpy and matplotlib


In [2]:
base_file = "/data/henry6/mindcontrol_ucsf_env/watchlists/long/VEO/antje_cohort/"

In [7]:
msid = "ms1049.txt"
msid_file = base_file + msid 

In [25]:

line = open(msid, "r").readlines()[1]

IndexError: list index out of range

In [82]:
for ms in msid:
    text_file = '/data/henry6/mindcontrol_ucsf_env/watchlists/long/VEO/antje_cohort/{}.txt'.format(ms)
    fread = pd.read_table(text_file,
                                 header=None)
    f_reversed = fread.iloc[::-1]
    mse_reversed = list(f_reversed[0])
    print("mse list is:", mse_reversed)

FileNotFoundError: File b'/data/henry6/mindcontrol_ucsf_env/watchlists/long/VEO/antje_cohort/m.txt' does not exist

In [ ]:
for ms in msid:
    text_file = '/data/henry6/mindcontrol_ucsf_env/watchlists/long/VEO/antje_cohort/{}.txt'.format(ms)
    fread = pd.read_table(text_file,
                                 header=None)
    f_reversed = fread.iloc[::-1]
    mse_reversed = list(f_reversed[0])
        print("mse list is:", mse_reversed)
        for mse_idx, mse in enumerate(mse_reversed):
            lst_edit_check = glob(os.path.join(outdir, mse, 'mindcontrol', '*FLAIR*',
                                               'lst', 'lst_edits', 'no_FP_filled_FN_dr2*'))
            if len(lst_edit_check) == 1:
                lesion_edit = ''.join(lst_edit_check)
                break
            elif len(lst_edit_check) > 1:
                raise ValueError("lst_edits files have more than one inputs, please check PBROUT directory",
                                 os.path.split(lst_edit_check[0]))

        try:
            print("The path of lsf lesion folder is:", lesion_edit)
            print("The lesion lst mse is:", mse)
        except NameError:
            print("The edited FLAIR lesion is not found in any timepoints, please check the corresponding directory.")
        mse_tp1, mse_tp2 = get_mseid(ms, mse_reversed, mse)
        print("mse_tp1 and mse_tp2 are:", mse_tp1, mse_tp2)
        if mse_tp1 is not '' and mse_tp2 is not '':
            if mse_tp1 == mse:
                check_after_edit_lesion(mse_tp1, mse_tp2, outdir, 5050, entry_types=["lst"])
                run_pbr_apply_transform(mse_tp2)
                check_before_mc_up(mse_tp2, outdir, 5050, entry_types=["lst"], lesion_mse=mse) #
                mc_up(mse_tp2)
                print("Done!")
            else:
                edit_lst(mse_tp1)
                check_after_edit_lesion(mse_tp1, mse_tp2, outdir, 5050, entry_types=["transform"])
                run_pbr_apply_transform(mse_tp2)
                check_before_mc_up(mse_tp2, outdir, 5050, entry_types=["transform"], lesion_mse=mse) #
                mc_up(mse_tp2)
                print("Done!")
        else:
            print("Either mse_tp1 or mse_tp2 is empty, or both of them are empty:", mse_tp1, mse_tp2)

In [50]:
with open(msid_file, "r") as f:
    tp1 = str(f.readlines()[0])
    print(tp1)
    if os.path.exists("/data/henry7/PBR/subjects/"+ tp1):
        print(tp1)
    with open(msid_file, "r") as foo:
        x = len(foo.readlines())
        print(x)
    
    #for nifti in os.listdir():
    
    
    
    

mse1827

8


In [76]:
TP1_base_dir = "/data/henry7/PBR/subjects/mse1827/nii/"
t1_TP1 = TP1_base_dir + "ms1049-mse1827-002-AX_T1_3D_IRSPGR.nii.gz"
t2_TP1 = TP1_base_dir + "ms1049-mse1827-003-FRFSE-001.nii.gz"

TP2_base_dir = "/data/henry7/PBR/subjects/mse1828/nii/"
t1_TP2 = TP2_base_dir+ "ms1049-mse1828-003-AX_T1_3D_IRSPGR.nii.gz"
t2_TP2 = TP2_base_dir+ "ms1049-mse1828-005-FRFSE-001.nii.gz"

mni_template = "/data/pelletier1/PDPD2/diffusion/standard/MNI152_T1_1mm.nii.gz"


In [75]:
reorient = Reorient2Std()
reorient.inputs.in_file = t1_TP1
reorient.inputs.out_file = TP1_base_dir + "t1_MNI_aligned.nii.gz"
res = reorient.run() 

In [77]:
#register t1_TP1 to MNI space - Done! 
flt = fsl.FLIRT()
flt.inputs.dof = 12
flt.inputs.in_file = t1_TP1
flt.inputs.reference = mni_template
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.out_matrix_file = TP1_base_dir+"t1_TP1_MNI.mat"
flt.inputs.out_file = TP1_base_dir+"t1_TP1_MNI.nii.gz"
print(flt.cmdline)
flt.run() 

flirt -in /data/henry7/PBR/subjects/mse1827/nii/ms1049-mse1827-002-AX_T1_3D_IRSPGR.nii.gz -ref /data/pelletier1/PDPD2/diffusion/standard/MNI152_T1_1mm.nii.gz -out /data/henry7/PBR/subjects/mse1827/nii/t1_TP1_MNI.nii.gz -omat /data/henry7/PBR/subjects/mse1827/nii/t1_TP1_MNI.mat -dof 12


In [79]:
#register t2_TP1 to T1 tp1 MNI space 
flt = fsl.FLIRT()
flt.inputs.dof = 6
flt.inputs.in_file = t2_TP1
flt.inputs.reference = TP1_base_dir + "t1_TP1_MNI.nii.gz"
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.out_matrix_file = TP1_base_dir+"t2_TP1_t1MNI.mat"
#flt.inputs.out_file = TP1_base_dir+"t2_TP1_t1MNI.nii.gz"
print(flt.cmdline)
flt.run() 

flirt -in /data/henry7/PBR/subjects/mse1827/nii/ms1049-mse1827-003-FRFSE-001.nii.gz -ref /data/henry7/PBR/subjects/mse1827/nii/t1_TP1_MNI.nii.gz -out /data/henry7/PBR/subjects/mse1827/nii/t2_TP1_t1MNI.nii.gz -omat /data/henry7/PBR/subjects/mse1827/nii/t2_TP1_t1MNI.mat -dof 6


In [80]:
# convert xfm - combine the two .mat files 
invt = fsl.ConvertXFM()
invt.inputs.in_file = TP1_base_dir+"t1_TP1_MNI.mat"
invt.inputs.in_file2 = TP1_base_dir+"t2_TP1_t1MNI.mat"
invt.inputs.concat_xfm = True
invt.inputs.out_file = TP1_base_dir+ "concat.mat"
invt.cmdline
invt.run()

In [81]:
flt = fsl.FLIRT()
flt.inputs.interp = "nearestneighbour"
flt.inputs.in_file = t2_TP1
flt.inputs.reference = TP1_base_dir+"t1_TP1_MNI.nii.gz"
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.apply_xfm 
flt.inputs.in_matrix_file = TP1_base_dir+ "concat.mat"
flt.inputs.out_file = TP1_base_dir + "t2TP1_t1TP1MNI.nii.gz"
print(flt.cmdline)
flt.run() 
                        

flirt -in /data/henry7/PBR/subjects/mse1827/nii/ms1049-mse1827-003-FRFSE-001.nii.gz -ref /data/henry7/PBR/subjects/mse1827/nii/t1_TP1_MNI.nii.gz -out /data/henry7/PBR/subjects/mse1827/nii/t2TP1_t1TP1MNI.nii.gz -omat ms1049-mse1827-003-FRFSE-001_flirt.mat -init /data/henry7/PBR/subjects/mse1827/nii/concat.mat -interp nearestneighbour


In [56]:
# register T2 tp2 tp T1 tp2 space 
flt = fsl.FLIRT()
flt.inputs.dof = 6
flt.inputs.in_file = t2_TP2
flt.inputs.reference = t1_TP2
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.out_matrix_file = TP2_base_dir+"t2TP1_t1TP2.mat"
flt.inputs.out_file = TP1_base_dir+"t2TP1_t1TP2.mat"
print(flt.cmdline)
flt.run() 

flirt -in /data/henry7/PBR/subjects/mse1828/nii/ms1049-mse1828-005-FRFSE-000.nii.gz -ref /data/henry7/PBR/subjects/mse1828/nii/ms1049-mse1828-003-AX_T1_3D_IRSPGR.nii.gz -out /data/henry7/PBR/subjects/mse1827/nii/t2TP1_t1TP2.mat -omat /data/henry7/PBR/subjects/mse1828/nii/t2TP1_t1TP2.mat -dof 6


TraitError: The trait 'out_file' of a FLIRTOutputSpec instance is an existing file name, but the path  '/data/henry7/PBR/subjects/mse1827/nii/t2TP1_t1TP2.mat' does not exist.
Interface FLIRT failed to run. 

In [ ]:
flt = fsl.FLIRT()
flt.inputs.interp = "nearestneighbour"
flt.inputs.in_file = t1_TP1 
flt.inputs.reference = mni_template
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.apply_xfm 
flt.inputs.in_matrix_file = TP1_base_dir + "t1_TP1_MNI.mat"
flt.inputs.out_file = TP1_base_dir + "t1TP1MNI.nii.gz"
print(flt.cmdline)
flt.run() 
           